In [ ]:
import pandas as pd

In [ ]:
data=pd.read_csv("downloads/train.csv")

In [ ]:
data.head(10)

## Cleaning the data


In [ ]:
data.isnull().sum()

<p>We can check that age, cabin and Embarked has null values hence we need to fix them</p>
<OL>
    <li>replace null age with median(age) of passengers.
</li>
    <li>
drop cabin column as it has too many null values
</li>
    <li>
we will keep embarked unchanged as it has no effect on results.</li>
</OL>

In [ ]:
data["Age"]=data["Age"].fillna(data["Age"].median())

In [ ]:
data= data.drop(columns=["Cabin"])

In [ ]:
gender_survival=data.groupby("Sex")["Survived"].mean()

In [ ]:
Survived_gender=data.groupby("Sex")["Survived"].sum()

In [ ]:
total_gender=data.groupby("Sex")["Survived"].count()

In [ ]:
print(Survived_gender)

In [ ]:
print(total_gender)

## Analysis keypoints
<UL>
    <li>
        Female survival rate was higher(74.2%) compared to males(18.9%).
    </li>
    <li>
        233 females out of 314 total females survived, while only 109 men survived out of 577 onboard.
    </li>
</UL>

In [ ]:
data["familysize"]=data["SibSp"]+data["Parch"]+1

In [ ]:
data.groupby("familysize")["Fare"].mean().reset_index()

## Analysis keypoints
<UL>
    <li>
        Families of 6 people paid highest average price \$73.72.
    </li>
    <li>
        Solo travelers paid lowest price per family i.e \$21.24.
    </li>
    <li>
        Trend shows that Families with high numbers of members paid higher price per family however price per person is lower. 
    </li>
</UL>

How did survival rates vary by age group?

In [1]:
print("this is sample code to analyze Titanic dataset")

this is sample code to analyze Titanic dataset
